# Start

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import panel as pn

pn.extension("ipywidgets", "katex", "mathjax")
import sys
from inspect import signature
from random import shuffle, uniform

import ipywidgets as ipw
from matplotlib.animation import FuncAnimation

# from matplotlib.ticker import MultipleLocator
from matplotlib.figure import Figure

print("Packages succesfully loaded ")

# Functions from cookbook

In [ ]:
def limit_answer(x):
    s = str(x)

    # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
    if not "." in s:
        n_decimal = 0
    else:
        n_decimal = len(s) - s.index(".") - 1

    range = 5 * 10 ** -(n_decimal + 1)

    return range


def check_nummeric_answers(
    id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt
):

    def button_callback(b):
        attempt.value += 1
        # print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = "Well done, this is correct!"

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W

            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = "Oops, there seems to be a mistake"

            if attempt.value >= 3:
                feedback_widget.value = (
                    "The correct answer is " + str(answer) + str(unit) + "."
                )

    return (
        button_callback  # otherwise gives TypeError: 'NoneType' object is not callable
    )


def nummeric_question_body(questions, units, answers, FB_good, FB_wrong):
    all_widgets = []
    attempts = []
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(
        questions, units, answers, FB_good, FB_wrong
    ):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width=750)
        unit_widget = pn.widgets.StaticText(value=units, width=10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width=100)
        # feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button = pn.widgets.Button(name="Check")

        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(
            check_nummeric_answers(
                id, answer, units, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt
            )
        )

    return all_widgets

In [ ]:
class class_variables:
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)


def classify_variables(locals, max_size_MB=0):
    max_size = max_size_MB * 1024 * 1024
    FV = class_variables()
    for key, value in {**globals(), **locals}.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV

In [ ]:
def add_local_variables_to(FV, locals, max_size_MB=0):
    max_size = max_size_MB * 1024 * 1024
    for key, value in locals.items():
        if sys.getsizeof(value) < max_size or max_size <= 0:
            FV.__setattr__(key, value)
    return FV


def check_code_values(IV):
    # The function variable (FV) is now defined as Input Variable (IV)
    # The input variable and the newly defined local variable will be merged to FV

    def get_coded_values(FV, GV):
        def button_callback(b):
            try:
                for i, param in enumerate(FV.check_parameters):

                    # get the value that students gave
                    response = getattr(GV, param)

                    # store them in the widget
                    FV.all_parameter_widgets[i].value = response

                FV.debug_widget.value = ""
            except:
                FV.debug_widget.value = (
                    "<b> Careful, not all parameters are defined! </b>"
                )

        return button_callback

    def check_coded_values(FV, GV):
        def button_callback(b):
            FV.attempt.value += 1

            for i, param in enumerate(FV.check_parameters):
                # response = getattr(GV, param)
                response = FV.all_parameter_widgets[i].value
                answer = getattr(FV, param)

                if np.abs(response - answer) < FV.f_margin * answer:
                    FV.all_feedback_widgets[i].value = "Nice, this is good!"

                if np.abs(response - answer) >= FV.f_margin * answer:
                    if FV.attempt.value < 3:
                        FV.all_feedback_widgets[i].value = (
                            "This one is incorrect, try again!"
                        )
                    if FV.attempt.value >= 3:
                        FV.all_feedback_widgets[i].value = (
                            "This one is incorrect, the answer should be "
                            + str(answer)
                            + "."
                        )

                # print('Debug: ', param, response, answer)

        return button_callback

    all_parameter_widgets = []
    info_widgets = []
    all_feedback_widgets = []
    for name, param in zip(IV.name_parameters, IV.check_parameters):
        symbol_widget = pn.widgets.StaticText(name="", value=name, width=100)
        parameter_widget = pn.widgets.FloatInput(name="", value=0, width=100)
        feedback_widget = pn.widgets.StaticText(value="")

        all_parameter_widgets.append(parameter_widget)
        all_feedback_widgets.append(feedback_widget)

        new_row = pn.Row(symbol_widget, parameter_widget, feedback_widget)
        info_widgets.append(new_row)

    debug_widget = pn.widgets.StaticText(value="")
    # attempt = pn.widgets.FloatInput(value=0)

    get_values_button = pn.widgets.Button(name="Load values")
    check_values_button = pn.widgets.Button(name="Check loaded values")

    GV = classify_variables(globals())
    FV = add_local_variables_to(IV, locals())

    # The error margin is set at 0.01% if it is not defined
    if "f_margin" not in FV.__dict__:
        FV.__setattr__("f_margin", 0.0001)

    get_values_button.on_click(get_coded_values(FV, GV))
    check_values_button.on_click(check_coded_values(FV, GV))

    row_buttons = pn.Row(get_values_button, check_values_button, debug_widget)
    Input_col = pn.Column(row_buttons, *info_widgets)
    display(Input_col)

In [ ]:
def check_code_function(
    fig, horizontal_axis, function_name, correct_function, par_x_axis, f_margin=0.001
):
    # The error margin (f_margin) is set at 0.1% if it is not defined

    # make the graph and show the correct answer
    ax = fig.subplots()
    pane = pn.pane.Matplotlib(fig, dpi=100)

    #  Plot the answer if the students function is found
    try:
        # Load the student-made function from globals
        function = globals()[function_name]

        # Add the arguments to the name of the function, for eval()
        # + Replace the argument on the x-axis so that it can be assessed with list comprehension.
        # https://docs.python.org/3/library/inspect.html
        # https://peps.python.org/pep-0362/
        sig = signature(function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        function = function_name + str(sig_function)
        title = function_name + str(sig)
        title = title.replace("_", " ")

        sig = signature(correct_function)
        sig_function = str(sig).replace(par_x_axis, "par_x_axis")
        correct_function2 = str(correct_function.__name__) + sig_function

        # calculate the student's answer through eval()
        student_answer = [eval(function) for par_x_axis in horizontal_axis]

        # calculate the correct answer
        correct_answer = []
        for par_x_axis in horizontal_axis:
            correct_answer.append(eval(correct_function2))
        # This code should work, but it does not
        # correct_answer = [eval(correct_function2) for par_x_axis in horizontal_axis]

        # check if the answer is correct and plot it before/below lines
        changes = np.array(student_answer) - np.array(correct_answer)
        inaccuracy = np.abs(1 - np.array(correct_answer) / np.array(student_answer))

        if np.max(changes) == 0:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Perfect!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        if np.max(changes) != 0 and np.max(inaccuracy) < f_margin:
            y_loc = (np.mean(correct_answer) + np.min(correct_answer)) / 2
            text = ax.text(
                np.mean([horizontal_axis]),
                y_loc,
                "Good!",
                fontsize=12,
                color="#1b5a00",
                ha="center",
                va="center",
            )

        # plot the answers
        line = ax.plot(horizontal_axis, student_answer, label="Your answer")
        line = ax.plot(horizontal_axis, correct_answer, label="Correct answer")
        ax.legend()
        ax.set_title(title)

    except:
        text_failed = "Almost there, \n your function can not be plotted, \n please try to fix the bug"
        x_ticks = ax.get_xticks()
        y_ticks = ax.get_yticks()
        text = ax.text(
            np.average(x_ticks),
            np.average(y_ticks),
            text_failed,
            fontsize=16,
            color="r",
            ha="center",
            va="center",
        )

    # update the graph
    display(pane)

# Commonly used functions

In [ ]:
def dispersion(k, h):  # calculate omega
    return (9.81 * k * np.tanh(k * h)) ** 0.5

In [ ]:
def wave_length(T, h):
    L = 9.81 * T**2 / (2 * np.pi)
    L_all = [L]

    for i in range(1500):
        L = 9.81 * T**2 / (2 * np.pi) * np.tanh(2 * np.pi * h / L)
        L_all.append(L)

        if np.abs(L_all[-1] - L_all[-2]) < 0.0005:
            # plt.plot(L_all)
            break

    return round(L, 13)


# wave_length(T = 3, h = 600)

In [ ]:
def wave_length_check(T, h):
    Dif = []
    L_serie = np.arange(5, 1000, 0.01)
    for L in L_serie:
        LHS = (2 * np.pi / T) ** 2
        RHS = 9.81 * 2 * np.pi / L * np.tanh(2 * np.pi / L * h)
        Dif.append(np.abs(LHS - RHS))
        # if len(Dif) > 2 and Dif[-1] > Dif[-2]:
        #    break

    id_best = np.argmin(Dif)
    L = L_serie[id_best]
    plt.plot(Dif)

    return round(L, 13)


# wave_length_check(T = 3,h = 600)

In [ ]:
def group_stats(k1, k2, w1, w2):
    Delta_k = np.abs(k2 - k1)
    Delta_w = np.abs(w2 - w1)
    L = 2 * np.pi / Delta_k
    T = 2 * np.pi / Delta_w
    cg = Delta_w / Delta_k
    return L, T, cg

# Student calculators

Worden niet meer gebruikt.

In [ ]:
def wavelength_calculator():
    # define widgets (with initial value for L)
    T = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)

    L_init = wave_length(T.value, h.value)
    L = pn.widgets.FloatInput(value=L_init, step=0.001, width=100, disabled=True)

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        L.value = wave_length(T.value, h.value)

    T.param.watch(update_L_widget, "value")
    h.param.watch(update_L_widget, "value")

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value="<b>Input</b>", width=200)
    title_output = pn.widgets.StaticText(value="<b>Output</b>", width=200)
    heading = pn.widgets.StaticText(value="<b>Wavelength calculator</b>")
    heading = pn.widgets.StaticText(
        value='<span style="font-size: 20px;"><b>Wavelength calculator</b></span>'
    )

    ## Add descriptions and width for alignment
    symbol_T = pn.widgets.StaticText(value="T", width=1)
    unit_T = pn.widgets.StaticText(value="s", width=1)
    T_widget = pn.Row(symbol_T, T, unit_T)

    symbol_h = pn.widgets.StaticText(value="h", width=1)
    unit_h = pn.widgets.StaticText(value="m", width=1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_L = pn.widgets.StaticText(value="L", width=1)
    unit_L = pn.widgets.StaticText(value="m", width=1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    # add latex formula
    text_formula = pn.widgets.StaticText(value="Solves iteratively:", width=100)
    formula = pn.pane.LaTeX(r"$L=\frac{gT^2}{2 \pi} tanh( \frac{2 \pi h}{L})$")
    row_formula = pn.Row(text_formula, formula)

    ## merge the layout and display the result
    input_widget = pn.Column(title_input, T_widget, h_widget)
    output_widget = pn.Column(title_output, L_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading, row_formula, horizontal_allignment)

    return include_heading


# wavelength_calculator()

In [ ]:
def angular_frequency_calculator():
    # define widgets (with initial value for L)
    L = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)

    w = pn.widgets.FloatInput(
        value=dispersion(2 * np.pi / L.value, h.value),
        step=0.001,
        width=100,
        disabled=True,
    )

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        w.value = dispersion(2 * np.pi / L.value, h.value)

    L.param.watch(update_L_widget, "value")
    h.param.watch(update_L_widget, "value")

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value="<b>Input</b>", width=200)
    title_output = pn.widgets.StaticText(value="<b>Output</b>", width=200)
    heading = pn.widgets.StaticText(
        value='<span style="font-size: 20px;"><b>Angular frequency calculator (w)</b></span>'
    )

    ## Add descriptions and width for alignment
    symbol_L = pn.widgets.StaticText(value="L", width=1)
    unit_L = pn.widgets.StaticText(value="m", width=1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    symbol_h = pn.widgets.StaticText(value="h", width=1)
    unit_h = pn.widgets.StaticText(value="m", width=1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_w = pn.widgets.StaticText(value="w", width=1)
    unit_w = pn.widgets.StaticText(value="m", width=1)
    w_widget = pn.Row(symbol_w, w, unit_w)

    # add latex formula
    text_formula = pn.widgets.StaticText(value="Solves:", width=100)
    formula = pn.pane.LaTeX(r"$\omega = \sqrt{ g k tanh(kh)}$")
    row_formula = pn.Row(text_formula, formula)

    ## merge the layout and display the result
    input_widget = pn.Column(title_input, L_widget, h_widget)
    output_widget = pn.Column(title_output, w_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading, row_formula, horizontal_allignment)

    return include_heading


# angular_frequency_calculator()

In [ ]:
def W2_calculators():
    allignment = pn.Row(wavelength_calculator(), angular_frequency_calculator())
    return allignment


# W2_calculators()

# Questions

### Q1

In [ ]:
def W2_Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = (
        "Can you asses the wavelength through an iterative approach when the wave period (T) is "
        + str(T1)
        + " seconds and the water depth (h) is "
        + str(h1)
        + " meter? Feel free to use the cell below to make the computations."
    )
    text_widget = pn.widgets.StaticText(value=text_general, width=750)
    text_widget = pn.widgets.StaticText(value=text_general)

    Q1_text = "a) What is the deep water wavelength (L0)?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = (
        "Indeed, the deep water wavelength is in this way related to the wave period."
    )
    Q1_FB_W = "There is a mistake, the only variable is the wave period."

    Q2_text = "b) If you use the dispersion relation expressed in the wavelength, what is the wavelength in the first iteration?"
    Q2_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi) * np.tanh(2 * np.pi * h1 / L)
    Q2_answer = round(L, 2)
    Q2_FB_G = ""
    Q2_FB_W = "Try to fill in the deep water wavelength for the first iteration."

    Q3_text = "c) To what wavelength does this iteration converge?"
    Q3_unit = " m"
    Q3_answer = round(wave_length(T1, h1), 2)
    Q3_FB_G = ""
    Q3_FB_W = "There is a mistake, "

    questions = [Q1_text, Q2_text, Q3_text]
    units = [Q1_unit, Q2_unit, Q3_unit]
    answers = [Q1_answer, Q2_answer, Q3_answer]
    FB_good = [Q1_FB_G, Q2_FB_G, Q3_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W, Q3_FB_W]

    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)

    display(pn.Column(text_widget, *all_widgets))


# W2_Q1()

### Q2

The formating code makes the arrays vertical, so the code is a bit harder to read. The arrays are columns from the book Coastal Dynamics

In [ ]:
def W2_Q2():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    # T1 = 6.7
    # h1 = 4.5

    text_general = (
        "Can you asses the wavelength through table B-3 of the book, when the wave period (T) is "
        + str(T1)
        + " seconds and the water depth (h) is "
        + str(h1)
        + " meter? Feel free to use the cell below to make the computations."
    )
    text_widget = pn.widgets.StaticText(value=text_general, width=750)
    text_widget = pn.widgets.StaticText(value=text_general)

    Q1_text = "a) What is the ratio of h/L0? (Use 3 decimal numbers in this question)"
    Q1_unit = ""
    L0 = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(h1 / L0, 3)
    Q1_FB_G = ""
    Q1_FB_W = ""

    Q2_text = "b) What is the ratio of h/L, you can linear interpolate. (Use 3 decimal numbers in this question)"
    Q2_unit = ""

    # data from table B-3, page 536, of the book CD1
    h_L0_serie = np.array(
        [
            0,
            0.002,
            0.004,
            0.006,
            0.008,
            0.01,
            0.015,
            0.02,
            0.025,
            0.03,
            0.035,
            0.04,
            0.045,
            0.05,
            0.055,
            0.06,
            0.065,
            0.07,
            0.075,
            0.08,
            0.085,
            0.09,
            0.095,
            0.1,
            0.11,
            0.12,
            0.13,
            0.14,
            0.15,
            0.16,
            0.17,
            0.18,
            0.19,
            0.2,
            0.21,
            0.22,
            0.23,
            0.24,
            0.25,
            0.26,
            0.27,
            0.28,
            0.29,
            0.30,
            0.31,
            0.32,
            0.33,
            0.34,
            0.35,
            0.36,
            0.37,
            0.38,
            0.39,
            0.4,
            0.41,
            0.42,
            0.43,
            0.44,
            0.45,
            0.46,
            0.47,
            0.48,
            0.49,
            0.5,
            1,
        ]
    )
    h_L_serie = np.array(
        [
            0,
            0.0179,
            0.0253,
            0.0311,
            0.0360,
            0.0403,
            0.0496,
            0.0576,
            0.0648,
            0.0713,
            0.0775,
            0.0833,
            0.0888,
            0.0942,
            0.0993,
            0.104,
            0.109,
            0.114,
            0.119,
            0.123,
            0.128,
            0.132,
            0.137,
            0.141,
            0.150,
            0.158,
            0.167,
            0.175,
            0.183,
            0.192,
            0.2,
            0.208,
            0.217,
            0.225,
            0.234,
            0.242,
            0.251,
            0.259,
            0.268,
            0.277,
            0.285,
            0.294,
            0.303,
            0.312,
            0.321,
            0.330,
            0.339,
            0.349,
            0.358,
            0.367,
            0.377,
            0.386,
            0.395,
            0.405,
            0.415,
            0.424,
            0.434,
            0.433,
            0.453,
            0.463,
            0.472,
            0.482,
            0.492,
            0.502,
            1,
        ]
    )

    # read the table
    id_lower = np.where(h_L0_serie <= h1 / L0)[0][
        -1
    ]  # get the (last) id where the value is below h/L0
    h_L_lower = h_L_serie[
        id_lower
    ]  # use this id to get the new ratio H/L, the lower boundary
    h_L_upper = h_L_serie[id_lower + 1]  # the upper boundary

    # linear interpolate
    slope = (h_L_upper - h_L_lower) / (h_L0_serie[id_lower + 1] - h_L0_serie[id_lower])
    h_L = h_L_lower + slope * (h1 / L0 - h_L0_serie[id_lower])

    # print(h1/L0, id_lower, h_L_lower, h_L_upper, h_L)
    Q2_answer = round(h_L, 3)
    Q2_FB_G = ""
    Q2_FB_W = ""

    Q3_text = "c) What is the wavelength?"
    Q3_unit = " m"
    Q3_answer = round(h1 / h_L, 2)
    Q3_FB_G = ""
    Q3_FB_W = ""

    Q4_text = "d) What is the value tanh(kh), you can linearly interpolate. (Use 3 decimal numbers in this question)"
    Q4_unit = ""

    # data from table B-3
    tanh_kh_serie = np.array(
        [
            0,
            0.112,
            0.158,
            0.193,
            0.222,
            0.248,
            0.302,
            0.347,
            0.386,
            0.420,
            0.452,
            0.48,
            0.507,
            0.531,
            0.554,
            0.575,
            0.595,
            0.614,
            0.632,
            0.649,
            0.665,
            0.681,
            0.695,
            0.681,
            0.695,
            0.709,
            0.735,
            0.759,
            0.780,
            0.8,
            0.818,
            0.835,
            0.85,
            0.864,
            0.877,
            0.888,
            0.899,
            0.909,
            0.918,
            0.926,
            0.933,
            0.940,
            0.946,
            0.952,
            0.957,
            0.961,
            0.965,
            0.969,
            0.972,
            0.975,
            0.978,
            0.980,
            0.983,
            0.984,
            0.986,
            0.988,
            0.989,
            0.990,
            0.991,
            0.992,
            0.993,
            0.994,
            0.995,
            0.995,
            0.996,
            0.996,
            1,
        ]
    )

    # read the table
    id_lower = np.where(h_L0_serie <= h1 / L0)[0][
        -1
    ]  # get the (last) id where the value is below h/L0
    tanh_kh_lower = tanh_kh_serie[id_lower]
    tanh_kh_upper = tanh_kh_serie[id_lower + 1]

    # linear interpolate
    slope = (tanh_kh_upper - tanh_kh_lower) / (
        h_L0_serie[id_lower + 1] - h_L0_serie[id_lower]
    )
    tanh_kh = tanh_kh_lower + slope * (h1 / L0 - h_L0_serie[id_lower])

    Q4_answer = round(tanh_kh, 3)
    Q4_FB_G = ""
    Q4_FB_W = ""

    questions = [Q1_text, Q2_text, Q3_text, Q4_text]
    units = [Q1_unit, Q2_unit, Q3_unit, Q4_unit]
    answers = [Q1_answer, Q2_answer, Q3_answer, Q4_answer]
    FB_good = [Q1_FB_G, Q2_FB_G, Q3_FB_G, Q4_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W, Q3_FB_W, Q4_FB_W]

    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)

    display(pn.Column(text_widget, *all_widgets))

    # print(h1 / L0)
    # print(h_L)
    # print(h_L_lower)
    # print(h_L_upper)
    # print(tanh_kh_lower)
    # print(tanh_kh_upper)


# W2_Q2()

### Q3

In [ ]:
def W2_Q3():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    T1 = 6.7
    h1 = 4.5

    text_general = (
        "A third method to calculate the wave length is through the implicit formula of Fenton, when the wave period (T) is "
        + str(T1)
        + " seconds and the water depth (h) is "
        + str(h1)
        + " meter? Feel free to use the cell below to make the computations."
    )
    text_widget = pn.widgets.StaticText(value=text_general, width=750)
    text_widget = pn.widgets.StaticText(value=text_general)

    def waveNumber_Fenton(T, d):  # made by J.A.ArriagaGarcia (Jaime)
        g = 9.81
        omega = 2 * np.pi / T
        k0 = omega * omega / g
        alpha = k0 * d
        beta = alpha * (np.tanh(alpha)) ** -0.5
        k = (
            (alpha + beta**2 * np.cosh(beta) ** -2)
            / (np.tanh(beta) + beta * np.cosh(beta) ** -2)
            / d
        )
        return k

    Q1_text = (
        "a) What is the wave number (k), following the explicit formula of Fentom?"
    )
    Q1_unit = ""

    k1 = waveNumber_Fenton(T=T1, d=h1)
    Q1_answer = round(k1, 2)
    Q1_FB_G = ""
    Q1_FB_W = ""

    Q2_text = "b) What is the wave length, following the explicit formula of Fentom?"
    Q2_unit = " m"
    Q2_answer = round(2 * np.pi / k1, 2)
    Q2_FB_G = ""
    Q2_FB_W = ""

    questions = [Q1_text, Q2_text]
    units = [Q1_unit, Q2_unit]
    answers = [Q1_answer, Q2_answer]
    FB_good = [Q1_FB_G, Q2_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W]

    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)

    display(pn.Column(text_widget, *all_widgets))


# W2_Q3()

In [ ]:
def nummeric_question_body(
    questions, units, answers, FB_good, FB_wrong, random_order=False
):
    all_widgets = []
    attempts = []

    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = (
            questions[i],
            units[i],
            answers[i],
            FB_good[i],
            FB_wrong[i],
        )
        id = i + 1
        question_widget = pn.widgets.StaticText(value=question, width=750)
        unit_widget = pn.widgets.StaticText(value=unit, width=10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width=100)
        # feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button = pn.widgets.Button(name="Check")

        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(
            check_nummeric_answers(
                id, answer, unit, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt
            )
        )

    return all_widgets

### Q4

In [ ]:
def W2_Q4():

    h = round(uniform(2, 4), 1)  # m , random value from 2 to 4 (excluding 4)
    L = round(uniform(25, 30), 1)  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h)

    Q1_text = (
        "What is the wave period (T) for a wave with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q1_unit = " s"
    Q1_answer = T = round(2 * np.pi / omega, 2)
    Q1_FB_G = ""
    Q1_FB_W = ""

    h = round(uniform(2, 4), 1)  # m
    L = round(uniform(20, 30), 1)  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h)
    T = 2 * np.pi / omega

    Q2_text = (
        "What is the wave celerity (c) for waves with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q2_unit = " m/s"
    Q2_answer = c = round(L / T, 2)
    Q2_FB_G = ""
    Q2_FB_W = ""

    h = round(uniform(1, 2), 1)
    L = round(uniform(20, 30), 1)  # m
    omega = dispersion(k, h)
    T = 2 * np.pi / omega

    Q3_text = (
        "What is the wave celerity (c) for waves with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q3_unit = " m/s"
    Q3_answer = c = round(L / T, 2)
    Q3_FB_G = ""
    Q3_FB_W = ""

    h = round(uniform(1, 2), 1)
    T = round(uniform(5, 7), 1)
    L = wave_length(T, h)
    Q4_text = (
        "What is the wave length (L) for a wave with a period (T) of "
        + str(T)
        + " seconds when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q4_unit = " m"
    Q4_answer = round(L, 2)
    Q4_FB_G = ""
    Q4_FB_W = ""

    h = round(uniform(5, 7), 1)  # km
    L = 500000  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h * 1000)  # m to km
    T = 2 * np.pi / omega
    Q5_text = (
        "What is the propagation speed (c) of a tsunami wave train with (L) of 500 km in the deep ocean with a depth (h) of "
        + str(h)
        + " km?"
    )
    Q5_unit = " km/h"
    Q5_answer = c = round(L / T / 1000 * 3600, 2)
    Q5_FB_G = ""
    Q5_FB_W = ""

    h = round(uniform(5, 7), 1)  # km
    T = round(uniform(30, 38), 0)  # min
    L = wave_length(T * 60, h)
    Q6_text = (
        "What is the propagation speed of a tsunami wave train with a period (T) of "
        + str(T)
        + " minutes in the deep ocean with a depth (h) of "
        + str(h)
        + " km?"
    )
    Q6_unit = " km/h"
    Q6_answer = c = round(L / (T) / 1000 * 60, 2)
    Q6_FB_G = ""
    Q6_FB_W = ""

    h = round(uniform(0.6, 10), 1)
    T = round(uniform(2.5, 12), 1)
    L = round(wave_length(T, h), 1)  # releastic value for question, based on T

    k = 2 * np.pi / L
    n = 0.5 + k * h / np.sinh(2 * k * h)
    Q7_text = (
        "What is the value of n when the water depth (h) is "
        + str(h)
        + " m, and the wave length (L) is "
        + str(L)
        + " m?"
    )
    Q7_unit = ""
    Q7_answer = round(n, 2)
    Q7_FB_G = ""
    Q7_FB_W = ""

    h = round(uniform(0.8, 10), 1)
    T = round(uniform(2.5, 12), 1)
    L = wave_length(T, h)
    k = 2 * np.pi / L
    c = L / T
    n = 0.5 + k * h / np.sinh(2 * k * h)
    cg = n * c

    Q8_text = (
        "What is the group velocity (cg) when the water depth (h) is "
        + str(h)
        + " m, and the wave period (T) is "
        + str(T)
        + " s?"
    )
    Q8_unit = " m/s"
    Q8_answer = round(cg, 2)
    Q8_FB_G = ""
    Q8_FB_W = ""

    # get realistic value
    h = round(uniform(1, 10), 1)
    T = round(uniform(4, 8), 1)
    L = round(wave_length(T, h), 1)

    omega = dispersion(k, h)
    T = 2 * np.pi / omega
    k = 2 * np.pi / L
    c = L / T
    n = 0.5 + k * h / np.sinh(2 * k * h)
    cg = n * c

    Q9_text = (
        "At which speed does the wave energy propagate when the water depth (h) is "
        + str(h)
        + " m, and the wave length (L) is "
        + str(L)
        + " m?"
    )
    Q9_unit = " m/s"
    Q9_answer = round(cg, 2)
    Q9_FB_G = ""
    Q9_FB_W = ""

    questions = [
        Q1_text,
        Q2_text,
        Q3_text,
        Q4_text,
        Q5_text,
        Q6_text,
        Q7_text,
        Q8_text,
        Q9_text,
    ]
    units = [
        Q1_unit,
        Q2_unit,
        Q3_unit,
        Q4_unit,
        Q5_unit,
        Q6_unit,
        Q7_unit,
        Q8_unit,
        Q9_unit,
    ]
    answers = [
        Q1_answer,
        Q2_answer,
        Q3_answer,
        Q4_answer,
        Q5_answer,
        Q6_answer,
        Q7_answer,
        Q8_answer,
        Q9_answer,
    ]
    FB_good = [
        Q1_FB_G,
        Q2_FB_G,
        Q3_FB_G,
        Q4_FB_G,
        Q5_FB_G,
        Q6_FB_G,
        Q7_FB_G,
        Q8_FB_G,
        Q9_FB_G,
    ]
    FB_wrong = [
        Q1_FB_W,
        Q2_FB_W,
        Q3_FB_W,
        Q4_FB_W,
        Q5_FB_W,
        Q6_FB_W,
        Q7_FB_W,
        Q8_FB_W,
        Q9_FB_W,
    ]

    all_widgets = nummeric_question_body(
        questions, units, answers, FB_good, FB_wrong, random_order=True
    )

    display(pn.Column(*all_widgets))


# W2_Q4()

### Q5

In [ ]:
def W2_Q5():
    h1 = uniform(0.5, 2)
    h2 = uniform(0.5, 3.5)
    h3 = uniform(0.5, 5)
    h4 = uniform(2, 5)
    h5 = uniform(0.3, 0.91)

    L1 = h1 / uniform(0.025, 0.05)  # shallow water
    L2 = h2 / uniform(0.051, 0.49)  # intermediate water
    L3 = h3 / uniform(0.5, 0.75)  # deep water
    L4 = h4 / uniform(0.025, 0.075)  # shallow or intermediate water
    L5 = h5 / uniform(0.25, 0.75)  # intermediate water or deep water

    L_serie = [L1, L2, L3, L4, L5]
    h_serie = [h1, h2, h3, h4, h5]

    # The possible answers (may very per question)
    answers = ["Shallow", "Intermediate", "Deep"]

    Questions = []
    correct_answers_id = []

    for L, h in zip(L_serie, h_serie):
        L = round(L, 2)
        h = round(h, 2)
        Questions.append("L = " + str(L) + " m, h = " + str(h) + " m")

        ratio = h / L

        if ratio <= 0.05:
            correct_answers_id.append(0)
        if ratio > 0.05 and ratio < 0.5:
            correct_answers_id.append(1)
        if ratio >= 0.5:
            correct_answers_id.append(2)

    # define the widgets for visualization, make for each a row with question and answer
    # Store all the rows with statements in a list for visualization, and in toggle_widgets for checking the answer
    Rows = []
    toggle_widgets = []
    for i in range(len(correct_answers_id)):
        question_widget = pn.widgets.StaticText(
            value=Questions[i], width=150
        )  # statement

        radio_group_widget = pn.widgets.RadioButtonGroup(
            name="Radio Button Group", options=answers, button_type="default"
        )
        toggle_widgets.append(radio_group_widget)

        add_row = pn.Row(question_widget, radio_group_widget)
        Rows.append(add_row)

    # randomize the order of statements
    shuffle(Rows)

    # add a submit button with a feedback option next to it
    submit_button = pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
    submit_row = pn.Row(submit_button, feedback_widget)

    # include a question
    text_general = "Select if the wave described on the left experiences shallow, intermediate, or deep water."
    text_widget = pn.widgets.StaticText(value=text_general)

    # structure the widgets and display thems
    display(text_widget, *Rows, submit_row)

    # check the answer and give feedback
    def check_answers(button):
        score = 0

        for i in range(len(correct_answers_id)):
            if toggle_widgets[i].value == answers[correct_answers_id[i]]:
                score += 1

        # print(toggle_widget.value)
        feedback_widget.value = (
            "Your score is " + str(score) + "/" + str(len(correct_answers_id))
        )

    submit_button.on_click(check_answers)


# W2_Q5()

### Q6

In [ ]:
from matplotlib.figure import Figure

In [ ]:
def check_answers_W2_Q6(
    id,
    answers,
    unit,
    FB_G,
    FB_W,
    num_widgets,
    feedback_widgets,
    attempt,
    final_score_widget,
    plots,
    panes,
    figures,
    par,
):
    FV = par
    omega_serie, T_serie, L_serie, k_serie, c_serie, n_serie, cg_serie = (
        FV["omega_serie"],
        FV["T_serie"],
        FV["L_serie"],
        FV["k_serie"],
        FV["c_serie"],
        FV["n_serie"],
        FV["cg_serie"],
    )

    def button_callback(b):
        attempt.value += 1

        # store responses in a list for plotting
        responses = []
        score = 0
        for i in range(len(answers)):
            num_widget = num_widgets[i]
            feedback_widget = feedback_widgets[i]
            answer = answers[i]
            response = num_widget.value
            responses.append(response)

            # if answer is correct
            if np.abs(answer - num_widget.value) < limit_answer(answer):
                score += 1
                if len(FB_G) != 0:
                    feedback_widget.value = FB_G
                else:
                    feedback_widget.value = "Well done, this is correct!"

            # the answer is NOT within boundaries, provide feedback based on the number of attempts
            if np.abs(answer - num_widget.value) >= limit_answer(answer):

                if attempt.value < 3 and len(FB_W) > 0:
                    feedback_widget.value = FB_W

                if attempt.value < 3 and len(FB_W) == 0:
                    feedback_widget.value = "Oops, there seems to be a mistake"

                if attempt.value >= 3:
                    feedback_widget.value = (
                        "The correct answer is " + str(answer) + str(unit) + "."
                    )

        final_score_widget.value = str(score) + "/" + str(len(answers))

        # Update the graphs
        if id == 1:
            ax = plots[0]
            ax.clear()
            line = ax.plot(omega_serie, L_serie, label="L [m]")
            # ax.scatter(par['omega_points'], np.array(answers) * 1000, c="green", s=30, label = 'correct answer')
            ax.scatter(
                par["omega_points"],
                np.array([responses]) * 1000,
                c="#006AB5",
                s=25,
                label="answer",
            )
            ax.set_title(par["titles"][0])
            ax.legend()
            ax.set_xlabel("$\omega $ [rad/s]")
            panes[0].object = figures[0]

        if id == 2 or id == 3:
            # the number of times question 2 and 3 are answered
            attempt2 = par["attempts"][1].value
            attempt3 = par["attempts"][2].value

            # the given answers
            responses2 = return_answers_widgets(par["all_answers"][1])
            responses3 = return_answers_widgets(par["all_answers"][2])

            ax = plots[1]
            ax.clear()
            if attempt2 > 0:
                line = ax.plot(omega_serie, c_serie, label="c [m/s]", color="#006AB5")
                # ax.scatter(par['omega_points'], answers, color ="#006AB5", s=30, label = 'correct answer')
                ax.scatter(
                    par["omega_points"],
                    responses2,
                    color="#006AB5",
                    s=25,
                    label="answer c",
                )
                ax.set_title(par["titles"][1])
                ax.legend()
                ax.set_xlabel("$\omega $ [rad/s]")
                panes[1].object = figures[1]

            if attempt3 > 0:
                line = ax.plot(omega_serie, cg_serie, label="cg [m/s]", color="#003f6c")
                ax.scatter(
                    par["omega_points"],
                    responses3,
                    c="#003f6c",
                    s=25,
                    label="answer cg",
                )
                ax.set_title(par["titles"][1])
                ax.legend()
                ax.set_xlabel("$\omega $ [rad/s]")
                panes[1].object = figures[1]

        if id == 3:
            ax = plots[2]
            ax.clear()
            line = ax.plot(omega_serie, n_serie, label="n [-]")
            ax.set_title(par["titles"][2])
            ax.legend()
            ax.set_xlabel("$\omega $ [rad/s]")
            panes[2].object = figures[2]

    return (
        button_callback  # otherwise gives TypeError: 'NoneType' object is not callable
    )

In [ ]:
def return_answers_widgets(widgets):
    answers = []
    for widget in widgets:
        answers.append(widget.value)
    return answers

In [ ]:
def W2_Q6():
    h = np.arange(3000, 6000 + 500, 500)
    h = np.random.choice(h)

    T1 = 5 * 60
    T4 = 30 * 60

    text_general = (
        "We are going to analyze the characteristics of tsunami waves at a depth of "
        + str(h)
        + " m. It will be analyzed for waves with a period of "
        + str(T1 / 60)
        + " minutes (point 1), at the shallow water boundary (point 2), the deep water boundary (point 3), and for waves with a period of "
        + str(T4 / 60)
        + "  minutes (point 4)."
    )

    Q1 = "What is the wave length of the tsunami wave at those 4 points?"
    Q1_labels = ["L1", "L2", "L3", "L4"]
    Q1_unit = " km"

    L1 = wave_length(T=T1, h=h)
    L2 = h / 0.05
    L3 = h / 0.5
    L4 = wave_length(T=T4, h=h)
    Q1_answers = [round(L1 / 1000, 2), L2 / 1000, L3 / 1000, round(L4 / 1000, 2)]

    # The 4 points of interest in this question
    w1 = 2 * np.pi / T1
    w2 = dispersion(k=2 * np.pi / (L2), h=h)
    w3 = dispersion(k=2 * np.pi / (L3), h=h)
    w4 = 2 * np.pi / T4
    omega_points = np.array([w1, w2, w3, w4])

    Q2 = "What is the wave celerity (c) of the tsunami wave at those 4 points?"
    Q2_labels = ["c1", "c2", "c3", "c4"]
    Q2_unit = " m/s"
    T2 = 2 * np.pi / w2
    T3 = 2 * np.pi / w3

    def calc_c(T, h):
        L = wave_length(T, h)
        return 9.81 * T / (2 * np.pi) * np.tanh(2 * np.pi * h / L)

    c1 = L1 / T1
    c2 = L2 / T2
    c3 = L3 / T3
    c4 = L4 / T4

    c1 = calc_c(T1, h)
    c2 = calc_c(T2, h)
    c3 = calc_c(T3, h)
    c4 = calc_c(T4, h)

    Q2_answers = [round(c1, 2), round(c2, 2), round(c3, 2), round(c4, 2)]

    Q3 = "What is the wave group velocity (cg) of the tsunami at those 4 points?"
    Q3_labels = ["c_{g1}$", "c_{g2}", "c_{g3}", "c_{g4}"]
    Q3_unit = " m/s"

    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    k3 = 2 * np.pi / L3
    k4 = 2 * np.pi / L4

    n1 = 0.5 + k1 * h / np.sinh(2 * k1 * h)
    n2 = 0.5 + k2 * h / np.sinh(2 * k2 * h)
    n3 = 0.5 + k3 * h / np.sinh(2 * k3 * h)
    n4 = 0.5 + k4 * h / np.sinh(2 * k4 * h)

    cg1 = c1 * n1
    cg2 = c2 * n2
    cg3 = c3 * n3
    cg4 = c4 * n4

    Q3_answers = [round(cg1, 2), round(cg2, 2), round(cg3, 2), round(cg4, 2)]

    # store the questions in a list
    Questions = [Q1, Q2, Q3]
    Unit_question = [Q1_unit, Q2_unit, Q3_unit]
    answer_question = [Q1_answers, Q2_answers, Q3_answers]
    label_question = [Q1_labels, Q2_labels, Q3_labels]

    # no feedback provided
    FB_G, FB_W = "", ""

    def calc_c(T, h):
        L = wave_length(T, h)
        return 9.81 * T / (2 * np.pi) * np.tanh(2 * np.pi * h / L)

    # the answers for the graph
    omega_serie = np.linspace(w4, w3 * 2, 100)
    T_serie = [2 * np.pi / w for w in omega_serie]
    L_serie = [wave_length(T, h) for T in T_serie]
    k_serie = [2 * np.pi / L for L in L_serie]
    # c_serie = [L/T for L,T in zip(L_serie, T_serie)]
    c_serie = [calc_c(T, h) for T in T_serie]
    n_serie = [0.5 + k * h / np.sinh(2 * k * h) for k in k_serie]
    cg_serie = [c * n for c, n in zip(c_serie, n_serie)]

    # set plot settings and make plots
    titles = ["Wave length", "Celerity", "n"]
    figures = []
    plots = []
    panes = []
    for i, (title) in enumerate(titles):
        fig = Figure((5, 2.5))
        ax = fig.subplots()
        ax.set_axis_off()
        pane = pn.pane.Matplotlib(fig, dpi=96)
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(
            bottom=0.25
        )  # Add some extra space for the axis at the bottom
        fig.subplots_adjust(left=0.2)  # Add some space for the labels
        figures.append(fig)
        plots.append(ax)
        panes.append(pane)

    # fill an empty list with widgets
    all_question_widgets = []
    all_answers = []
    attempts = (
        []
    )  # widgets only used for counting how many times the submit button is pressed
    id = 0
    for i, (question, unit, answers, label) in enumerate(
        zip(Questions, Unit_question, answer_question, label_question)
    ):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width=750)

        Rows_answer = []
        num_widgets = []
        feedback_widgets = []
        for number, answer in enumerate(answers):
            number_widget = pn.widgets.StaticText(
                value=str(number + 1) + str(")"), width=10
            )
            unit_widget = pn.widgets.StaticText(value=unit, width=10)
            num_widget = pn.widgets.FloatInput(value=0, step=0.01, width=100)
            feedback_widget = pn.widgets.StaticText(value="", name="", width=250)

            num_widgets.append(num_widget)
            feedback_widgets.append(feedback_widget)

            Hbox = pn.Row(number_widget, num_widget, unit_widget, feedback_widget)
            Rows_answer.append(Hbox)
        all_answers.append(num_widgets)

        # Add a submit button with widget that returns the final score
        submit_button = pn.widgets.Button(name="Check")
        submit_text_widget = pn.widgets.StaticText(value="Final score:", width=70)
        final_score_widget = pn.widgets.TextInput(value="", name="", width=130)
        row_submit = pn.Row(submit_button, submit_text_widget, final_score_widget)

        # The values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)

        # store the local parameters to be used inside the function
        FV = locals()
        submit_button.on_click(
            check_answers_W2_Q6(
                id,
                answers,
                unit,
                FB_G,
                FB_W,
                num_widgets,
                feedback_widgets,
                attempt,
                final_score_widget,
                plots,
                panes,
                figures,
                FV,
            )
        )

        # Structure the widgets
        Vbox_answer = pn.Column(*Rows_answer, row_submit)
        Hbox_plot = pn.Row(Vbox_answer, panes[i])
        question_widget = pn.Column(question_widget, Hbox_plot)
        all_question_widgets.append(question_widget)

    text_widget = pn.widgets.StaticText(value=text_general)
    quiz_widget = pn.Column(text_widget, *all_question_widgets)

    return quiz_widget


# W2_Q6()

### Q7

In [ ]:
from ipywidgets import HBox, VBox, interact

In [ ]:
def dispersion(k, h):
    return (9.81 * k * np.tanh(k * h)) ** 0.5


def W2_Q7_graph(T1, T2, T3, slope_in, d0):
    %matplotlib inline
    # bed profile

    slope = 1.0 / slope_in  # bed slope [-]
    d0  # offshore water depth [m]
    x_max = round((d0 + 2) / slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    # x = np.linspace(0, x_max + x_max/100, 100)# <-- should be used to reduce computational demands, influences xticks
    zbed = -(d0 - slope * x)  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h_water = h[0:x0_id]
    x_water = x[0:x0_id]

    # wave length through profile
    L1 = [wave_length(T1, h) for h in h_water]
    L2 = [wave_length(T2, h) for h in h_water]
    L3 = [wave_length(T3, h) for h in h_water]

    # velocity profile
    def calc_c(T, h):
        L = wave_length(T, h)
        return 9.81 * T / (2 * np.pi) * np.tanh(2 * np.pi * h / L)

    c1 = [calc_c(T1, h) for h in h_water]
    c2 = [calc_c(T2, h) for h in h_water]
    c3 = [calc_c(T3, h) for h in h_water]

    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(9, 6), sharex=True, sharey=False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.1)

    # bathymetry
    axs[0].plot(x, zbed, label="Bed (1:" + str(round(slope_in, 2)) + ")", color="k")
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water surface")
    axs[0].set_ylabel("y [m]")
    axs[0].legend(loc="lower right")

    # wave length
    axs[1].plot(x_water, L1, label="wave 1")
    axs[1].plot(x_water, L2, label="wave 2")
    axs[1].plot(x_water, L3, label="wave 3")
    axs[1].set_ylim(0, np.max(([L1], [L2], [L3])) * 1.1)
    axs[1].set_ylabel("Wavelength (L) [m]")
    axs[1].legend(loc="upper right")

    # wave celerity
    axs[2].plot(x_water, c1, label="wave 1")
    axs[2].plot(x_water, c2, label="wave 2")
    axs[2].plot(x_water, c3, label="wave 3")
    axs[2].set_xlim(0, np.max(x))
    axs[2].set_ylim(0, np.max(([c1], [c2], [c3])) * 1.1)
    axs[2].set_ylabel("Celerity (c) [m/s]")
    axs[2].legend(loc="upper right")
    axs[2].set_xlabel("cross-shore location (x) [m]")
    # axs[2].set_xticks('cross-shore location (x) [m]')

    # remove the lines related to the x-ticks
    axs[0].xaxis.set_visible(False)
    axs[1].xaxis.set_visible(False)

    # set title
    axs[0].set_title("Wave characteristics in cross-shore direction")

    # get values of xticks, change them to get x=0 at water boundary and positive direction offshore
    xticks = axs[2].get_xticks()
    new_ticks = np.ones(len(xticks)) * x_water[-1] - xticks + 1
    axs[2].set_xticks(xticks, new_ticks)

    # Plot dots at the transition with intermediate water depth
    # for L,c in zip((L1,L2,L3), (c1,c2,c3)):
    #    h_L = h_water/L
    #    if h_L[0] > 0.5:
    #       id_deep = np.argwhere(h_L <= 0.5)[0][0]  # first location where water depth = 0
    #       axs[1].plot(x_water[id_deep], L[id_deep], 'ro', label="wave 3")
    #
    #    if h_L[0] > 0.05:
    #        id_shl = np.argwhere(h_L < 0.05)[0][0]  # first location where water depth = 0
    #        axs[1].plot(x_water[id_shl], L[id_shl], 'ro', label="wave 3")


def W2_Q7_set_graph(FV, FV2):

    def button_callback(b):
        if FV2["id"] == 1:
            FV["T1"].value = 10
            FV["T2"].value = 15
            FV["T3"].value = 20
            FV["slope"].value = 30
            FV["d0"].value = 250

        if FV2["id"] == 2:
            FV["T1"].value = 4
            FV["T2"].value = 6
            FV["T3"].value = 8
            FV["slope"].value = 30
            FV["d0"].value = 50

        if FV2["id"] == 3:
            FV["T1"].value = 4
            FV["T2"].value = 6
            FV["T3"].value = 8
            FV["slope"].value = 30
            FV["d0"].value = 20

    return button_callback


def W2_Q7_check_answers(FV, FV2):

    def button_callback(b):
        all_answers = FV2["all_answers"]
        response = FV2["checkbutton_group"].value

        if response == FV2["answer"]:
            FV2["feedback_widget"].value = FV2["FG"]
        else:
            FV2["feedback_widget"].value = FV2["FW"]

    return button_callback


def W2_Q7_questions(FV):
    all_answers = ["wave 1", "wave 2", "wave 3"]

    Q1 = "Which waves are in deep water at x = 6000?"
    Ans1 = [all_answers[0], all_answers[1]]
    FG_1 = "Indeed, these two waves are in deep water. Wave 3, with a period of 20 seconds, is influenced by bottom friction."
    FW_1 = (
        "Try another reasoning, what are the spatial changes for waves in deep water?"
    )

    Q2 = "Which of the waves are in intermediate water at x = 500?"
    Ans2 = [all_answers[1], all_answers[2]]
    FG_2 = "Good Job, this wave is the only wave that in intermediate water depth"
    FW_2 = "Almost, which waves are affected by the changing water depth?"

    Q3 = "Which of the waves are in shallow water at x = 200?"
    Ans3 = []
    FG_3 = "Indeed, all of the waves are in intermediate water, the waves are in shallow water around x = 50 m, where the celerity is only related to the water depth."
    FW_3 = "Which parameters are relevant for waves in shallow water?"

    number_of_questions = 3

    all_question_widgets = []
    for id in np.arange(1, number_of_questions + 1, 1):
        question = eval("Q" + str(id))
        answer = eval("Ans" + str(id))
        FG = eval("FG_" + str(id))
        FW = eval("FW_" + str(id))

        question_widget = pn.widgets.StaticText(value=question)
        set_button = pn.widgets.Button(name="Set graph")
        checkbutton_group = pn.widgets.CheckButtonGroup(
            name="Check Button Group", value=[], options=all_answers
        )
        submit_button = pn.widgets.Button(name="Check")
        feedback_widget = pn.widgets.StaticText(value="", name="")

        FV2 = {key: value for key, value in locals().items()}
        # FV2 = {key: value for key, value in {**globals(), **locals()}.items()}
        set_button.on_click(W2_Q7_set_graph(FV, FV2))
        submit_button.on_click(W2_Q7_check_answers(FV, FV2))

        question_row = pn.Row(set_button, checkbutton_group, submit_button)
        question_widget = pn.Column(question_widget, question_row, feedback_widget)
        all_question_widgets.append(question_widget)

    return all_question_widgets


def W2_Q7():
    # Create interactive widgets, which require IPY Widgets, widgets from panel do not work
    # L1 = pn.widgets.FloatSlider(name='Float Slider', start=0, end=3.141, step=0.01, value=1.57)
    T1 = ipw.FloatSlider(value=4, min=0.01, max=50, step=0.01, description="T1 [s]")
    T2 = ipw.FloatSlider(value=7, min=0.01, max=50, step=0.01, description="T2 [s]")
    T3 = ipw.FloatSlider(value=25, min=0.01, max=50, step=0.01, description="T3 [s]")

    slope = ipw.FloatSlider(
        value=30, min=0.1, max=50, step=0.1, description="slope 1:..."
    )
    d0 = ipw.FloatSlider(value=50, min=0.1, max=500, step=0.1, description="depth [m]")

    # Setup widget layout (User Interface) for the graph input
    vbox1 = ipw.VBox(
        [
            ipw.Label("Wave components", layout=ipw.Layout(align_self="center")),
            T1,
            T2,
            T3,
        ]
    )
    vbox2 = ipw.VBox(
        [ipw.Label("General", layout=ipw.Layout(align_self="center")), slope, d0]
    )
    UI = ipw.HBox([vbox1, vbox2])

    # Use the interactive function to update the plot
    graph = ipw.interactive_output(
        W2_Q7_graph, {"T1": T1, "T2": T2, "T3": T3, "slope_in": slope, "d0": d0}
    )

    FV = {key: value for key, value in locals().items()}
    questions = W2_Q7_questions(FV)

    intro = "In the following question we are going to assess the depth the waves experience by analyzing the graph above. You can answer the questions by first setting the graph, through the button. When the graph is fully loaded, you can select the appropriate waves and check the answer. Good luck!"
    intro_widget = pn.widgets.StaticText(value=intro)

    display(UI, graph, intro_widget, *questions)


# W2_Q7()

### Q8

In [ ]:
def W2_Q8():

    # define widgets
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a [m]")
    a2 = ipw.FloatText(value=0.5, min=0, max=20, step=0.01, description="a [m]")

    T1 = ipw.FloatText(value=8, min=0.01, max=250000, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=4, min=0.01, max=250000, step=0.01, description="T [s]")

    phi_1 = ipw.FloatText(
        value=0, min=-1, max=1, step=0.01, description="phi [2 pi rad]"
    )  #'$\phi$ [2 $\pi$ rad]')
    phi_2 = ipw.FloatText(
        value=0.25, min=-1, max=1, step=0.01, description="phi [2 pi rad]"
    )  #'$\phi$ [2 $\pi$ rad]')

    depth = ipw.FloatText(value=7.5, min=1, max=250, step=0.01, description="h [m]")
    xp = ipw.FloatText(value=0, min=0, step=0.1, description="x [m]")
    tp = ipw.FloatText(value=0, min=1, step=0.1, description="t [s]")

    # Setup widget layout (User Interface) and display
    vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), a1, T1, phi_1]
    )
    vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), a2, T2, phi_2]
    )
    vbox4 = ipw.VBox(
        [ipw.Label("General", layout=ipw.Layout(align_self="center")), depth, xp, tp]
    )

    ui = ipw.HBox([vbox1, vbox2, vbox4])

    def calc_eta(a1, T1, phi_1, a2, T2, phi_2, xp, tp, depth):
        n_waves = 3
        L1 = wave_length(T1, depth)
        L2 = wave_length(T2, depth)

        L_group, T_group, c_g = group_stats(
            k1=2 * np.pi / L1,
            k2=2 * np.pi / L2,
            w1=2 * np.pi / T1,
            w2=2 * np.pi / T2,
        )

        T = np.min([T1, T2])
        L = np.max([L1, L2])
        t = np.arange(0, n_waves * T_group + T / 30, T / 30)
        x = np.arange(0, n_waves * L_group + L / 30, L / 30)

        fig, axs = plt.subplots(
            nrows=3, ncols=2, figsize=(9, 5), sharex=False, sharey=False
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # time based
        ax1 = axs[0, 0]
        ax2 = axs[1, 0]
        ax3 = axs[2, 0]
        # space based
        ax4 = axs[0, 1]
        ax5 = axs[1, 1]
        ax6 = axs[2, 1]

        # calculate surface, including phase change
        eta1_T = a1 * np.sin(
            2 * np.pi / T1 * t - 2 * np.pi / L1 * xp - phi_1 * (2 * np.pi)
        )
        eta2_T = a2 * np.sin(
            2 * np.pi / T2 * t - 2 * np.pi / L2 * xp - phi_2 * (2 * np.pi)
        )
        eta_T = eta1_T + eta2_T

        eta1_x = a1 * np.sin(
            2 * np.pi / T1 * tp - 2 * np.pi / L1 * x - phi_1 * (2 * np.pi)
        )
        eta2_x = a2 * np.sin(
            2 * np.pi / T2 * tp - 2 * np.pi / L2 * x - phi_2 * (2 * np.pi)
        )
        eta_x = eta1_x + eta2_x

        # calculate surface, without phase change
        eta1_T_basic = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T_basic = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
        eta_T_basic = eta1_T_basic + eta2_T_basic

        eta1_x_basic = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_x_basic = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
        eta_x_basic = eta1_x_basic + eta2_x_basic

        # plot surface excluding phase change
        ax1.plot(t, eta1_T_basic, color="grey", linestyle="--", label="reference")
        ax2.plot(t, eta2_T_basic, color="grey", linestyle="--", label="reference")
        ax3.plot(t, eta_T_basic, color="grey", linestyle="--", label="$\eta_1$")
        ax4.plot(x, eta1_x_basic, color="grey", linestyle="--", label="reference")
        ax5.plot(x, eta2_x_basic, color="grey", linestyle="--", label="reference")
        ax6.plot(x, eta_x_basic, color="grey", linestyle="--", label="$\eta_1$")

        # plot surface including phase change
        ax1.plot(t, eta1_T, label="$\eta_1$")
        ax2.plot(t, eta2_T, label="$\eta_2$")
        ax3.plot(t, eta_T, label="$\eta_{1+2}$")

        ax4.plot(x, eta1_x, label="$\eta_1$")
        ax5.plot(x, eta2_x, label="$\eta_2$")
        ax6.plot(x, eta_x, label="$\eta_{1+2}$")

        # set vertical axis the same
        amp = (a1 + a2) * 1.1
        ax1.set_ylim(-amp, amp)
        ax2.set_ylim(-amp, amp)
        ax3.set_ylim(-amp, amp)
        ax4.set_ylim(-amp, amp)
        ax5.set_ylim(-amp, amp)
        ax6.set_ylim(-amp, amp)

        # set horizontal axis
        ax1.set_xlim(0, n_waves * T_group)
        ax2.set_xlim(0, n_waves * T_group)
        ax3.set_xlim(0, n_waves * T_group)
        ax4.set_xlim(0, n_waves * L_group)
        ax5.set_xlim(0, n_waves * L_group)
        ax6.set_xlim(0, n_waves * L_group)

        # set labels
        ax1.set_ylabel("$\eta_1$ [m]")
        ax2.set_ylabel("$\eta_2$ [m]")
        ax3.set_ylabel("$\eta_{1+2}$ [m]")

        ax3.set_xlabel("t/T_{group}")
        ax6.set_xlabel("x/L_{group}")

        # remove the lines related to the x-ticks and y-ticks
        ax1.xaxis.set_visible(False)
        ax2.xaxis.set_visible(False)
        ax4.yaxis.set_visible(False)
        ax5.yaxis.set_visible(False)
        ax6.yaxis.set_visible(False)

        # set scaled ticks
        if n_waves >= 1:
            ax3.set_xticks(np.arange(0, n_waves // 1 + 1, 1) * T_group)
            ax3.set_xticklabels(np.arange(0, n_waves // 1 + 1, 1))
            ax6.set_xticks(np.arange(0, n_waves // 1 + 1, 1) * L_group)
            ax6.set_xticklabels(np.arange(0, n_waves // 1 + 1, 1))

        else:  # 3 times when the scale is smaller than 1
            ax3.set_xticks([0, 0.5 * n_waves * T_group, n_waves * T_group])
            ax3.set_xticklabels([0, 0.5 * n_waves, n_waves])
            ax6.set_xticks([0, 0.5 * n_waves * L_group, n_waves * L_group])
            ax6.set_xticklabels([0, 0.5 * n_waves, n_waves])

        # remove x and y ticks
        ax4.set_xticklabels([], fontsize=0)
        ax5.set_xticklabels([], fontsize=0)

        ax4.set_yticklabels([], fontsize=0)
        ax5.set_yticklabels([], fontsize=0)
        ax6.set_yticklabels([], fontsize=0)

        # set title
        ax1.set_title("Time-based (x =" + str(xp) + " m)")
        ax4.set_title("Space-based (t =" + str(tp) + " s)")

        # plot legends
        legend1 = ax4.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))
        legend2 = ax5.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))
        legend3 = ax6.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))

        # ui = ipw.HBox([vbox1, vbox2, vbox3, vbox4])

    # update graph
    out = ipw.interactive_output(
        calc_eta,
        {
            "a1": a1,
            "T1": T1,
            "phi_1": phi_1,
            "a2": a2,
            "T2": T2,
            "phi_2": phi_2,
            "xp": xp,
            "tp": tp,
            "depth": depth,
        },
    )

    display(ui, out)


# W2_Q8()

## Wave groups

In [ ]:
def W2_wave_groups():
    from scipy.signal import hilbert

    # define widgets
    a1 = ipw.FloatText(value=1, min=0, max=20, step=0.01, description="a [m]")
    a2 = ipw.FloatText(value=1.5, min=0, max=20, step=0.01, description="a [m]")

    T1 = ipw.FloatText(value=7, min=0.01, max=250000, step=0.01, description="T [s]")
    T2 = ipw.FloatText(value=6.2, min=0.01, max=250000, step=0.01, description="T [s]")

    phi_1 = ipw.FloatText(
        value=0, min=-1, max=1, step=0.01, description="phi [2 pi rad]"
    )  #'$\phi$ [2 $\pi$ rad]')
    phi_2 = ipw.FloatText(
        value=0, min=-1, max=1, step=0.01, description="phi [2 pi rad]"
    )  #'$\phi$ [2 $\pi$ rad]')

    n_waves = ipw.FloatText(value=3, min=0.1, max=10, step=0.1, description="n_{waves}")

    depth = ipw.FloatText(value=20, min=1, max=250, step=0.01, description="h [m]")
    xp = ipw.FloatText(value=0, min=0, step=0.1, description="x [m]")
    tp = ipw.FloatText(value=0, min=1, step=0.1, description="t [s]")

    L1 = ipw.FloatText(
        value=wave_length(T1.value, depth.value), description="L [m]", disabled=True
    )
    L2 = ipw.FloatText(
        value=wave_length(T2.value, depth.value), description="L [m]", disabled=True
    )
    Lgroup = ipw.FloatText(
        value=group_stats(
            k1=2 * np.pi / L1.value,
            k2=2 * np.pi / L2.value,
            w1=2 * np.pi / T1.value,
            w2=2 * np.pi / T2.value,
        )[0],
        description="L_{group} [m]",
        disabled=True,
    )

    c1 = ipw.FloatText(value=L1.value / T1.value, description="c [m/s]", disabled=True)
    c2 = ipw.FloatText(value=L2.value / T2.value, description="c [m/s]", disabled=True)

    h_L1 = ipw.FloatText(
        value=depth.value / L1.value, description="h/L [m]", disabled=True
    )
    h_L2 = ipw.FloatText(
        value=depth.value / L2.value, description="h/L [m]", disabled=True
    )

    # Setup widget layout (User Interface)
    vbox1 = ipw.VBox(
        [ipw.Label("Wave 1", layout=ipw.Layout(align_self="center")), a1, T1, c1, h_L1]
    )
    vbox2 = ipw.VBox(
        [ipw.Label("Wave 2", layout=ipw.Layout(align_self="center")), a2, T2, c2, h_L2]
    )
    vbox3 = ipw.VBox(
        [
            ipw.Label("Wave group", layout=ipw.Layout(align_self="center")),
            n_waves,
            Lgroup,
        ]
    )
    vbox4 = ipw.VBox(
        [ipw.Label("General", layout=ipw.Layout(align_self="center")), depth, xp, tp]
    )

    ui = ipw.HBox([vbox1, vbox2, vbox4])

    def calc_eta(a1, T1, phi_1, a2, T2, phi_2, L1, L2, n_waves, xp, tp, depth):
        L1 = wave_length(T1, depth)
        L2 = wave_length(T2, depth)

        L_group, T_group, c_g = group_stats(
            k1=2 * np.pi / L1,
            k2=2 * np.pi / L2,
            w1=2 * np.pi / T1,
            w2=2 * np.pi / T2,
        )

        T = np.min([T1, T2])
        L = np.max([L1, L2])
        # requires additional x and t values to get a correct Hilbert transformation at the graph boundaries
        # t = np.arange(0,n_waves*T_group+T/30,T/30)
        # x = np.arange(0,n_waves*L_group+L/30,L/30)
        t = np.arange(-0.5 * n_waves * T_group, (n_waves + 0.5) * T_group, T / 30)
        x = np.arange(-0.5 * n_waves * L_group, (n_waves + 0.5) * L_group, L / 30)

        fig, axs = plt.subplots(
            nrows=3, ncols=2, figsize=(9, 5), sharex=False, sharey=False
        )
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(wspace=0.06)

        # time based
        ax1 = axs[0, 0]
        ax2 = axs[1, 0]
        ax3 = axs[2, 0]
        # space based
        ax4 = axs[0, 1]
        ax5 = axs[1, 1]
        ax6 = axs[2, 1]

        # calculate surface, including phase change
        eta1_T = a1 * np.sin(
            2 * np.pi / T1 * t - 2 * np.pi / L1 * xp - phi_1 * (2 * np.pi)
        )
        eta2_T = a2 * np.sin(
            2 * np.pi / T2 * t - 2 * np.pi / L2 * xp - phi_2 * (2 * np.pi)
        )
        eta_T = eta1_T + eta2_T

        eta1_x = a1 * np.sin(
            2 * np.pi / T1 * tp - 2 * np.pi / L1 * x - phi_1 * (2 * np.pi)
        )
        eta2_x = a2 * np.sin(
            2 * np.pi / T2 * tp - 2 * np.pi / L2 * x - phi_2 * (2 * np.pi)
        )
        eta_x = eta1_x + eta2_x

        # calculate surface, without phase change
        eta1_T_basic = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T_basic = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
        eta_T_basic = eta1_T_basic + eta2_T_basic

        eta1_x_basic = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_x_basic = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
        eta_x_basic = eta1_x_basic + eta2_x_basic

        # calculate hilbert
        eta_T_envelope = np.abs(hilbert(eta_T_basic))
        eta_x_envelope = np.abs(hilbert(eta_x_basic))

        # carier wave
        k_bar = 2 * np.pi / L1 + 2 * np.pi / L2
        w_bar = (2 * np.pi / T1 + 2 * np.pi / T2) / 2
        car_wave_t = (a1 + a2) * np.sin(w_bar * t - k_bar * xp)
        car_wave_x = (a1 + a2) * np.sin(w_bar * tp - k_bar * x)

        # variable amplitude
        Delta_k = 2 * np.pi / L1 - 2 * np.pi / L2  # Delta k
        Delta_w = 2 * np.pi / T1 - 2 * np.pi / T2
        var_amp_t = (a1 + a2) * np.cos(Delta_w / 2 * t - Delta_k / 2 * xp)
        var_amp_x = (a1 + a2) * np.cos(Delta_w / 2 * tp - Delta_k / 2 * x)

        # plot surface excluding phase change
        ax3.plot(t, eta_T_basic, color="grey", linestyle="--", label="$\eta_1$")

        # plot surface including phase change
        ax1.plot(t, eta1_T, label="$\eta_1$")
        ax2.plot(t, eta2_T, label="$\eta_2$")
        ax3.plot(t, eta_T, label="$\eta_{1+2}$")
        ax3.plot(t, eta_T_envelope, label="Envelope")
        # ax3.plot(t,np.abs(var_amp_t), label = 'abs: car wave')

        ax4.plot(x, eta1_x, label="$\eta_1$")
        ax5.plot(x, eta2_x, label="$\eta_2$")
        ax6.plot(x, eta_x, label="$\eta_{1+2}$")
        ax6.plot(x, eta_x_envelope, label="Envelope")
        # ax6.plot(x,np.abs(var_amp_x), label = 'abs: car wave')

        # set vertical axis the same
        amp = (a1 + a2) * 1.1
        ax1.set_ylim(-amp, amp)
        ax2.set_ylim(-amp, amp)
        ax3.set_ylim(-amp, amp)
        ax4.set_ylim(-amp, amp)
        ax5.set_ylim(-amp, amp)
        ax6.set_ylim(-amp, amp)

        # set horizontal axis
        ax1.set_xlim(0, n_waves * T_group)
        ax2.set_xlim(0, n_waves * T_group)
        ax3.set_xlim(0, n_waves * T_group)
        ax4.set_xlim(0, n_waves * L_group)
        ax5.set_xlim(0, n_waves * L_group)
        ax6.set_xlim(0, n_waves * L_group)

        # set labels
        ax1.set_ylabel("$\eta_1$ [m]")
        ax2.set_ylabel("$\eta_2$ [m]")
        ax3.set_ylabel("$\eta_{1+2}$ [m]")

        ax3.set_xlabel("t/T_{group}")
        ax6.set_xlabel("x/L_{group}")

        # remove the lines related to the x-ticks and y-ticks
        ax1.xaxis.set_visible(False)
        ax2.xaxis.set_visible(False)
        ax4.yaxis.set_visible(False)
        ax5.yaxis.set_visible(False)
        ax6.yaxis.set_visible(False)

        # set scaled ticks
        if n_waves >= 1:
            ax3.set_xticks(np.arange(0, n_waves // 1 + 1, 1) * T_group)
            ax3.set_xticklabels(np.arange(0, n_waves // 1 + 1, 1))
            ax6.set_xticks(np.arange(0, n_waves // 1 + 1, 1) * L_group)
            ax6.set_xticklabels(np.arange(0, n_waves // 1 + 1, 1))

        else:  # 3 times when the scale is smaller than 1
            ax3.set_xticks([0, 0.5 * n_waves * T_group, n_waves * T_group])
            ax3.set_xticklabels([0, 0.5 * n_waves, n_waves])
            ax6.set_xticks([0, 0.5 * n_waves * L_group, n_waves * L_group])
            ax6.set_xticklabels([0, 0.5 * n_waves, n_waves])

        # remove x and y ticks
        ax4.set_xticklabels([], fontsize=0)
        ax5.set_xticklabels([], fontsize=0)

        ax4.set_yticklabels([], fontsize=0)
        ax5.set_yticklabels([], fontsize=0)
        ax6.set_yticklabels([], fontsize=0)

        # set title
        ax1.set_title("Time-based (x =" + str(xp) + " m)")
        ax4.set_title("Space-based (t =" + str(tp) + " s)")

        # plot legends
        legend1 = ax4.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))
        legend2 = ax5.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))
        legend3 = ax6.legend(loc="center left", bbox_to_anchor=(1.001, 0.5))

    # update graph
    out = ipw.interactive_output(
        calc_eta,
        {
            "a1": a1,
            "T1": T1,
            "phi_1": phi_1,
            "a2": a2,
            "T2": T2,
            "phi_2": phi_2,
            "L1": L1,
            "L2": L2,
            "n_waves": n_waves,
            "xp": xp,
            "tp": tp,
            "depth": depth,
        },
    )

    display(ui, out)


# W2_wave_groups()

## Q9 Coding

In [ ]:
def W2_Q9():
    # the question-related parameters
    T1 = round(uniform(5, 7), 1)
    T2 = round(uniform(7, 10), 1)
    h = 20

    # The question that is asked
    question = (
        "Can you complete the code below when the periods of wave 1 and 2 are "
        + str(T1)
        + " and "
        + str(T2)
        + " seconds and the water depth is "
        + str(h)
        + " m? You can check your computation after running the code and loading the values. Please use the iterative approach to calculate the wave length."
    )

    # The calculation
    L1 = wave_length(T1, h)
    L2 = wave_length(T2, h)

    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    w1 = 2 * np.pi / T1
    w2 = 2 * np.pi / T2

    Delta_k = np.abs(k2 - k1)
    Delta_w = np.abs(w2 - w1)
    k_average = np.average([k1, k2])
    w_average = np.average([w1, w2])

    c1 = L1 / T1
    c2 = L2 / T2

    c_average = np.average([c1, c2])
    L_group, T_group, c_group = group_stats(k1, k2, w1, w2)

    # Required widgets for functionality, does not have to be changed
    attempt_A = pn.widgets.FloatInput(value=0)
    attempt_B = pn.widgets.FloatInput(value=0)

    question_widget = pn.widgets.StaticText(name="", value=question)
    display(question_widget)

    # define a new global variable, so return (and related print) is prevented,
    # + a required parameter to count the attempt, dont change the name.
    global W2_Q9_param
    # store the question-related parameters and
    W2_Q9_param = (
        T1,
        T2,
        h,
        Delta_k,
        Delta_w,
        k_average,
        w_average,
        c_average,
        L_group,
        T_group,
        c_group,
        attempt_A,
        attempt_B,
    )


def Check_W2_Q9A():
    (
        T1,
        T2,
        h,
        Delta_k,
        Delta_w,
        k_average,
        w_average,
        c_average,
        L_group,
        T_group,
        c_group,
        attempt_A,
        attempt_B,
    ) = W2_Q9_param
    attempt = attempt_A

    # define the parameter names that have to be checked
    check_parameters = ["Delta_k", "Delta_w", "k_average", "w_average"]

    # define the names of the parameters as they are displayed
    name_parameters = ["Delta k", "Delta w", "k average", "w average"]

    # build  the question
    FV = classify_variables(locals())
    check_code_values(FV)


def Check_W2_Q9B():
    (
        T1,
        T2,
        h,
        Delta_k,
        Delta_w,
        k_average,
        w_average,
        c_average,
        L_group,
        T_group,
        c_group,
        attempt_A,
        attempt_B,
    ) = W2_Q9_param
    attempt = attempt_B

    # define the parameter names that have to be checked
    check_parameters = ["c_average", "c_group", "L_group", "T_group"]

    # define the names of the parameters as they are displayed
    name_parameters = ["cg", "c group", "L group", "T group"]

    # build the question
    FV = classify_variables(locals())
    check_code_values(FV)

## Q10

In [ ]:
def W2_Q10():

    question_start = "Can you use formula 2.4.3 to complete the functions below to plot the water elevation? "

    try:
        (
            T1,
            T2,
            h,
            Delta_k,
            Delta_w,
            k_average,
            w_average,
            c_average,
            L_group,
            T_group,
            c_group,
            attempt_A,
            attempt_B,
        ) = W2_Q9_param
        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)
        question_data = (
            "You can use the conditions from above, with wave periods T1 and T2 of "
            + str(T1)
            + " and "
            + str(T2)
            + " seconds, and a water depth of "
            + str(h)
            + " meter. "
        )

    except:
        # the question-related parameters
        T1 = round(uniform(5, 7), 1)
        T2 = round(uniform(7, 10), 1)
        h = 20

        L1 = wave_length(T1, h)
        L2 = wave_length(T2, h)
        L_group, T_group, c_group = group_stats(
            k1=2 * np.pi / L1, k2=2 * np.pi / L2, w1=2 * np.pi / L1, w2=2 * np.pi / L2
        )

        question_data = (
            "The wave periods T1 and T2 are "
            + str(T1)
            + " and "
            + str(T2)
            + " seconds, and the water depth is "
            + str(h)
            + " meter. "
        )

    # the amplitudes, where the amplitude of wave 2 is a factor 15% to 100% larger
    a1 = round(uniform(0.5, 1.5), 1)
    f_a2 = uniform(1.15, 2)
    a2 = round(a1 * f_a2, 1)
    question_data2 = (
        "The amplitudes of the waves are "
        + str(a1)
        + " and "
        + str(a2)
        + " meter, respectively. "
    )

    # the maximum coordinate, 3 times the wave group length, rounded to 25 meters
    x_max = 3 * L_group // 25 * 25 + 25

    # point of interest, near the end of the left wave group, rounded down with a stepsize of 25.
    xp = L_group // 25 * 25

    # maximum time, 3 times the wave group period
    t_max = 3 * T_group // 10 * 10 + 10

    # time point of interest
    tp = T_group // 10 * 10

    question_end = " The answer you coded will be plotted when you run the cell and the function is valid."

    question_boundaries = (
        "One function is used to plot time series at x = "
        + str(xp)
        + " m from t = 0 to t = "
        + str(t_max)
        + " seconds, and one to plot the wave conditions from x=0 to x="
        + str(x_max)
        + " m at t="
        + str(tp)
        + " seconds."
    )
    question = (
        question_start
        + question_data
        + question_data2
        + question_boundaries
        + question_end
    )

    # The question that is asked

    # make the attempt counter, one for each subquestion
    attempt = pn.widgets.FloatInput(value=0)

    # Required widgets for functionality, should not be changed
    question_widget = pn.widgets.StaticText(name="", value=question)
    display(question_widget)

    # define a new global variable, with a unique name (This is related to week 2, question 9)
    global W2_Q10_param
    # store the question-related parameters and the widget 'attempt'
    W2_Q10_param = a1, a2, T1, T2, L1, L2, x_max, xp, t_max, tp, L_group, T_group


# uncomment to use information provided by question 9.
# print('question 9')
# W2_Q9()

# print('question 10')
# W2_Q10()

In [ ]:
def Show_Q10A():
    # define the name of the function that the students will make
    function_name = "plot_eta_t"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "t"

    # set the horizontal axis of the graph
    a1, a2, T1, T2, L1, L2, x_max, xp, t_max, tp, L_group, T_group = W2_Q10_param
    horizontal_axis = np.arange(0, t_max + T_group / 30, T_group / 100)

    # define the correct function and its values along the x-axis.
    def correct_function(a1, a2, T1, T2, L1, L2, t, xp):
        eta1_T_basic = a1 * np.sin(2 * np.pi / T1 * t - 2 * np.pi / L1 * xp)
        eta2_T_basic = a2 * np.sin(2 * np.pi / T2 * t - 2 * np.pi / L2 * xp)
        eta_T_basic = eta1_T_basic + eta2_T_basic
        return eta_T_basic

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    fig = Figure((5, 2.5))
    check_code_function(
        fig,
        horizontal_axis,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
    )

In [ ]:
def Show_Q10B():
    # define the name of the function that the students will make
    function_name = "plot_eta_x"

    # define the name of the parameter plotted on the horizontal axis
    parameter_x_axis = "x"

    # set the horizontal axis of the graph
    a1, a2, T1, T2, L1, L2, x_max, xp, t_max, tp, L_group, T_group = W2_Q10_param
    horizontal_axis = np.arange(0, x_max + L_group / 30, L_group / 60)

    # define the correct function and its values along the x-axis.
    def correct_function(a1, a2, T1, T2, L1, L2, tp, x):
        eta1_T_basic = a1 * np.sin(2 * np.pi / T1 * tp - 2 * np.pi / L1 * x)
        eta2_T_basic = a2 * np.sin(2 * np.pi / T2 * tp - 2 * np.pi / L2 * x)
        eta_T_basic = eta1_T_basic + eta2_T_basic
        return eta_T_basic

    # set the acceptable computational error (ratio)
    f_margin = 0.001  # 0.001 = 0.01%

    fig = Figure((5, 2.5))
    check_code_function(
        fig,
        horizontal_axis,
        function_name,
        correct_function,
        parameter_x_axis,
        f_margin,
    )

## Q11 Animation (upcoming)